In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

## Import dependencies

In [3]:
import os
import pandas as pd
from IPython.display import display
from tqdm import tqdm

from configs import config
from configs.constants import ModelSetting
from utils.analyzers.subgroups_variance_analyzer import SubgroupsVarianceAnalyzer
from utils.common_helpers import create_tuned_base_model, save_metrics_to_file
from utils.custom_classes.data_loader import CompasWithoutSensitiveAttrsDataset
from utils.custom_classes.generic_pipeline import GenericPipeline
from utils.analyzers.subgroups_statistical_bias_analyzer import SubgroupsStatisticalBiasAnalyzer

## Configs

In [4]:
DATASET_NAME = "COMPAS_Without_Sensitive_Attributes"
EXPERIMENT_NAME = 'Hypothesis_Space'

SEX_priv = 1
RACE_priv = 'Caucasian'
N_ESTIMATORS = 200
SENSITIVE_ATTRIBUTES = ['sex', 'race']
PRIV_VALUES = [SEX_priv, RACE_priv]
TUNED_PARAMS_FILE_PATH = os.path.join('..', '..', 'results', 'models_tuning', f'tuning_results_{DATASET_NAME}_20230116__155240.csv')

## Models tuned hyper-parameters

In [5]:
models_tuned_params_df = pd.read_csv(TUNED_PARAMS_FILE_PATH)
models_tuned_params_df

,Dataset_Name,Model_Name,F1_Score,Accuracy_Score,Model_Best_Params
0,COMPAS_Without_Sensitive_Attributes,LogisticRegression,0.6785,0.6837,"{'max_iter': 50, 'penalty': 'l2', 'solver': 'n..."
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,0.6135,0.6165,"{'criterion': 'gini', 'max_depth': 5, 'max_fea..."
2,COMPAS_Without_Sensitive_Attributes,RandomForestClassifier,0.6944,0.6989,"{'max_depth': 4, 'max_features': 0.6, 'min_sam..."
3,COMPAS_Without_Sensitive_Attributes,XGBClassifier,0.6984,0.7027,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti..."
4,COMPAS_Without_Sensitive_Attributes,KNeighborsClassifier,0.6946,0.6960,"{'metric': 'manhattan', 'n_neighbors': 15, 'we..."


## Load dataset

In [6]:
dataset = CompasWithoutSensitiveAttrsDataset(dataset_path='../../data/COMPAS.csv')
dataset.X_data.head()

,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_F,c_charge_degree_M
0,0.0,-2.340451,1.0,-15.010999,1,0,0,0,1
1,0.0,0.000000,0.0,0.000000,1,0,0,1,0
2,0.0,0.000000,0.0,0.000000,0,0,1,1,0
3,0.0,0.000000,0.0,6.000000,1,0,0,0,1
4,0.0,0.000000,0.0,7.513697,1,0,0,1,0


## Run experiments

In [7]:
def create_base_pipeline(dataset, sensitive_attributes, priv_values, model_seed):
    base_pipeline = GenericPipeline(dataset, sensitive_attributes, priv_values)
    _ = base_pipeline.create_train_test_split_without_sensitive_attrs(dataset, config.TEST_SET_FRACTION, seed=model_seed)

    print('\nProtected groups splits:')
    for g in base_pipeline.test_groups.keys():
        print(g, base_pipeline.test_groups[g].shape)

    return base_pipeline


def get_model_metrics(base_model, n_estimators, dataset, sensitive_attributes, priv_values, model_seed,
                      dataset_name, base_model_name, exp_num=1):
    base_pipeline = create_base_pipeline(dataset, sensitive_attributes, priv_values, model_seed)
    print('\n\nX train and validation set: ')
    display(base_pipeline.X_train_val.head(10))

    # Compute variance metrics for subgroups
    save_results = False
    stability_fairness_analyzer = SubgroupsVarianceAnalyzer(ModelSetting.BATCH, n_estimators, base_model, base_model_name,
                                                            base_pipeline.X_train_val, base_pipeline.y_train_val,
                                                            base_pipeline.X_test, base_pipeline.y_test,
                                                            base_pipeline.sensitive_attributes, base_pipeline.priv_values, base_pipeline.test_groups,
                                                            base_pipeline.target, dataset_name)

    y_preds, variance_metrics_df = stability_fairness_analyzer.compute_metrics(save_results=save_results,
                                                                               result_filename=None,
                                                                               save_dir_path=None,
                                                                               make_plots=False)

    # Compute bias metrics for subgroups
    bias_analyzer = SubgroupsStatisticalBiasAnalyzer(base_pipeline.X_test, base_pipeline.y_test,
                                                     base_pipeline.sensitive_attributes, base_pipeline.priv_values,
                                                     base_pipeline.test_groups)
    dtc_res = bias_analyzer.compute_subgroups_metrics(y_preds,
                                                      save_results=False,
                                                      result_filename=None,
                                                      save_dir_path=None)
    bias_metrics_df = pd.DataFrame(dtc_res)

    # Save metrics
    metrics_df = pd.concat([variance_metrics_df, bias_metrics_df])
    result_filename = f'{EXPERIMENT_NAME}_Metrics_{dataset_name}_Experiment_{exp_num}_{base_model_name}'
    save_dir_path = os.path.join('..', '..', 'results', 'hypothesis_space')
    save_metrics_to_file(metrics_df, result_filename, save_dir_path)

    return metrics_df


In [8]:
def run_experiment(dataset, exp_num: int, model_seed: int):
    """
    Find variance and bias metrics for each model in config.MODELS_CONFIG.
    Save results in results/config.MODELS_CONFIG folder.

    :param exp_num: the number of experiment; is used to name the result file with metrics
    """
    for model_idx in tqdm(range(len(config.MODELS_CONFIG))):
        print('#' * 30, f' [Experiment {exp_num}] Analyze {config.MODELS_CONFIG[model_idx]["model_name"]} ', '#' * 30)
        model_seed += 1
        try:
            base_model = create_tuned_base_model(config.MODELS_CONFIG[model_idx]['model'],
                                                 config.MODELS_CONFIG[model_idx]['model_name'],
                                                 models_tuned_params_df)
            results_df = get_model_metrics(base_model, N_ESTIMATORS, dataset, SENSITIVE_ATTRIBUTES, PRIV_VALUES,
                                           model_seed=model_seed,
                                           dataset_name=DATASET_NAME,
                                           base_model_name=config.MODELS_CONFIG[model_idx]['model_name'],
                                           exp_num=exp_num)
            print(f'\n[Experiment {exp_num}] Metrics confusion matrix:')
            display(results_df)
        except Exception as err:
            print(f'ERROR with {config.MODELS_CONFIG[model_idx]["model_name"]}: ', err)

        print('\n\n\n')


### Experiment 1

In [9]:
# TOD: add dataset as a parameter
run_experiment(dataset, exp_num=1, model_seed=100)

  0%|          | 0/5 [00:00<?, ?it/s]

##############################  [Experiment 1] Analyze LogisticRegression  ##############################
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)

Protected groups splits:
sex_race_priv (339, 11)
sex_race_dis (111, 11)
sex_priv (857, 11)
sex_dis (199, 11)
race_priv (427, 11)
race_dis (629, 11)


X train and validation set: 


,age_cat_Greater than 45_0,c_charge_degree_F_1,c_charge_degree_M_0,priors_count,age_cat_Greater than 45_1,age_cat_25 - 45_1,age_cat_Less than 25_0,age_cat_Less than 25_1,juv_misd_count,age_cat_25 - 45_0,juv_other_count,juv_fel_count,c_charge_degree_F_0,c_charge_degree_M_1
470,1,1,1,0.106154,0,1,1,0,-0.127240,0,-0.148179,-0.098697,0,0
1328,1,1,1,2.409132,0,1,1,0,0.348324,0,-0.148179,-0.098697,0,0
3551,1,0,0,0.620185,0,1,1,0,-0.127240,0,-0.148179,-0.098697,1,1
1628,1,0,0,-0.085761,0,1,1,0,-0.127240,0,-0.148179,-0.098697,1,1
4138,1,1,1,-0.280851,0,1,1,0,-0.127240,0,1.111816,-0.098697,0,0
5030,1,1,1,0.693041,0,1,1,0,-0.127240,0,-0.148179,-0.098697,0,0
4510,1,0,0,-0.085761,0,1,1,0,-0.127240,0,-0.148179,-0.098697,1,1
1676,1,1,1,-0.661506,0,0,0,1,-0.127240,1,-0.148179,-0.098697,0,0
5047,1,0,0,-0.469591,0,1,1,0,-0.127240,0,-0.148179,-0.098697,1,1
1554,0,0,0,-0.668317,1,0,1,0,-0.127240,1,-0.148179,-0.098697,1,1


2023-01-16 18:08:13 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 1 / 200
2023-01-16 18:08:13 abstract_overall_variance_analyzer.py INFO    : Classifier 1 / 200 was tested
2023-01-16 18:08:13 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 2 / 200
2023-01-16 18:08:13 abstract_overall_variance_analyzer.py INFO    : Classifier 2 / 200 was tested
2023-01-16 18:08:13 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 3 / 200
2023-01-16 18:08:13 abstract_overall_variance_analyzer.py INFO    : Classifier 3 / 200 was tested
2023-01-16 18:08:13 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 4 / 200
2023-01-16 18:08:14 abstract_overall_variance_analyzer.py INFO    : Classifier 4 / 200 was tested
2023-01-16 18:08:14 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 5 / 200
2023-01-16 18:08:14 abstract_overall_variance_analyzer.py INFO    : Classifier 5 / 200 w



##############################  Stability metrics  ##############################
General Ensemble Accuracy: 0.643
Mean: 0.5254
Std: 0.0226
IQR: 0.0302
Entropy: 0.0
Jitter: 0.0364
Per sample accuracy: 0.6426
Label stability: 0.9543



[Experiment 1] Metrics confusion matrix:


...

,age_cat_Greater than 45_0,c_charge_degree_F_1,c_charge_degree_M_0,priors_count,age_cat_Greater than 45_1,age_cat_25 - 45_1,age_cat_Less than 25_0,age_cat_Less than 25_1,juv_misd_count,age_cat_25 - 45_0,juv_other_count,juv_fel_count,c_charge_degree_F_0,c_charge_degree_M_1
597,1,0,0,0.295811,0,1,1,0,-0.133157,0,-0.146069,-0.100065,1,1
3079,0,0,0,1.860390,1,0,1,0,-0.133157,1,-0.146069,-0.100065,1,1
414,1,1,1,-0.083087,0,1,1,0,-0.133157,0,-0.146069,-0.100065,0,0
936,1,0,0,2.287894,0,0,0,1,-0.133157,1,-0.146069,-0.100065,1,1
391,0,1,1,-3.115974,1,0,1,0,-0.133157,1,-0.146069,-0.100065,0,0
2494,0,0,0,-0.651433,1,0,1,0,-0.133157,1,-0.146069,-0.100065,1,1
3709,1,1,1,-0.651433,0,0,0,1,-0.133157,1,-0.146069,-0.100065,0,0
1102,1,1,1,0.106362,0,1,1,0,-0.133157,0,-0.146069,-0.100065,0,0
3241,0,1,1,-0.461984,1,0,1,0,-0.133157,1,-0.146069,-0.100065,0,0
124,1,1,1,-0.272535,0,1,1,0,-0.133157,0,-0.146069,-0.100065,0,0


2023-01-16 18:12:01 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 1 / 200
2023-01-16 18:12:01 abstract_overall_variance_analyzer.py INFO    : Classifier 1 / 200 was tested
2023-01-16 18:12:01 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 2 / 200
2023-01-16 18:12:01 abstract_overall_variance_analyzer.py INFO    : Classifier 2 / 200 was tested
2023-01-16 18:12:01 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 3 / 200
2023-01-16 18:12:01 abstract_overall_variance_analyzer.py INFO    : Classifier 3 / 200 was tested
2023-01-16 18:12:01 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 4 / 200
2023-01-16 18:12:01 abstract_overall_variance_analyzer.py INFO    : Classifier 4 / 200 was tested
2023-01-16 18:12:01 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 5 / 200
2023-01-16 18:12:01 abstract_overall_variance_analyzer.py INFO    : Classifier 5 / 200 w



##############################  Stability metrics  ##############################
General Ensemble Accuracy: 0.6884
Mean: 0.5387
Std: 0.1171
IQR: 0.162
Entropy: 0.3003
Jitter: 0.1914
Per sample accuracy: 0.6526
Label stability: 0.7306



[Experiment 1] Metrics confusion matrix:


,overall,sex_race_priv,sex_race_dis,sex_priv,sex_dis,race_priv,race_dis
General_Ensemble_Accuracy,0.688447,0.687879,0.680851,0.688811,0.686869,0.688940,0.688103
Mean,0.538719,0.602068,0.536376,0.529157,0.580152,0.606297,0.491566
Std,0.117054,0.113966,0.119373,0.117021,0.117200,0.114270,0.118997
IQR,0.162042,0.156869,0.167376,0.160878,0.167088,0.159255,0.163987
Entropy,0.300261,0.262233,0.000000,0.296586,0.000000,0.271286,0.320478
Jitter,0.191448,0.164676,0.217273,0.188377,0.204754,0.171569,0.205318
Per_Sample_Accuracy,0.652637,0.654818,0.628404,0.653054,0.650833,0.658721,0.648392
Label_Stability,0.730559,0.769697,0.685745,0.735921,0.707323,0.759424,0.710418
TPR,0.610656,0.496296,0.551724,0.630952,0.485294,0.482759,0.681529
TNR,0.755282,0.820513,0.738462,0.744292,0.792308,0.826923,0.694805


100%|██████████| 5/5 [04:39<00:00, 55.89s/it]